In [2]:
import pandas as pd
import numpy as np
import chart_studio.plotly as py
import cufflinks as cf
cf.go_offline(connected=True)

from pykrx import stock

In [8]:
data = pd.read_csv('data.csv', encoding='cp949')
data = data.set_index('종목코드')
data = data.rename_axis(index='티커')
data = data[['종목명', '종가']]
data.head()

,종목명,종가
티커,,
060310,3S,2885
054620,APS,7310
265520,AP시스템,20800
211270,AP위성,16120
139050,BF랩스,7730


In [4]:
df = stock.get_market_fundamental_by_ticker(date='20240315', market="KOSDAQ")
df.head()

,BPS,PER,PBR,EPS,DIV,DPS
티커,,,,,,
060310,947,96.17,3.05,30,0.00,0
054620,10864,14.48,0.67,505,0.00,0
265520,17980,3.81,1.16,5463,1.30,270
211270,5997,56.17,2.69,287,0.43,70
139050,5010,0.00,1.54,0,0.00,0


In [5]:
df.describe() #per<5, pbr<0.67

,BPS,PER,PBR,EPS,DIV,DPS
count,1698.000000,1698.000000,1698.000000,1698.000000,1698.000000,1698.000000
mean,6394.591873,35.337385,2.939823,572.537102,0.742303,86.898704
std,9783.363365,278.113901,14.070972,1411.139293,1.470416,270.090579
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1305.250000,0.000000,0.670000,0.000000,0.000000,0.000000
50%,3481.000000,5.995000,1.230000,86.000000,0.000000,0.000000
75%,7622.750000,20.262500,2.510000,564.750000,0.980000,70.000000
max,171357.000000,8700.000000,522.350000,21470.000000,14.210000,5100.000000


In [6]:
df2 = df.loc[(df['PER'] < 5) & (df['PBR'] < 0.67)]
df2 = df2[['PER', 'PBR']]
df2

,PER,PBR
티커,,
035760,0.00,0.45
023460,0.00,0.52
456440,0.00,0.00
290120,1.57,0.44
050120,1.37,0.32
...,...,...
192410,0.00,0.32
205470,1.17,0.60
115160,0.00,0.44


In [11]:
df3 = pd.merge(data, df2, on='티커')
df3

,종목명,종가,PER,PBR
티커,,,,
035760,CJ ENM,76300,0.00,0.45
023460,CNH,1787,0.00,0.52
456440,DB금융스팩11호,2270,0.00,0.00
290120,DH오토리드,3250,1.57,0.44
050120,ES큐브,2635,1.37,0.32
...,...,...,...,...
192410,휴림네트웍스,424,0.00,0.32
205470,휴마시스,1711,1.17,0.60
115160,휴맥스,2845,0.00,0.44


In [13]:
df3 = df3.loc[(df['PER'] > 0) & (df['PBR'] > 0)]
df3.shape #64개의 항목을 추출합니다

(64, 4)

In [129]:
data = stock.get_market_ohlcv('20240215', '20240315', ticker)
data = data[['종가']]
std_dev = data['종가'].rolling(20).std()

data['중심 밴드'] = data['종가'].rolling(20).mean()
data['이격도'] = (data['종가'] / data['중심 밴드']) * 100
data.dropna()

,종가,중심 밴드,이격도
날짜,,,
2024-03-14,2555,2793.25,91.470509
2024-03-15,2555,2775.50,92.055485


In [14]:
df3.index

Index(['290120', '050120', '151860', '040610', '016250', '121440', '035080',
       '036800', '091590', '039560', '032190', '017650', '007680', '005710',
       '140520', '067990', '088910', '013120', '038060', '012700', '241770',
       '101330', '053700', '023600', '037460', '017480', '038540', '093920',
       '011370', '035890', '011560', '024830', '115570', '036710', '037760',
       '352700', '115480', '127710', '054800', '013310', '306040', '003800',
       '073540', '054940', '009780', '073560', '032940', '030530', '012620',
       '377460', '024800', '119850', '024880', '033290', '052330', '124560',
       '104480', '081150', '106240', '225590', '003380', '023760', '045100',
       '205470'],
      dtype='object', name='티커')

In [15]:
result_df = pd.DataFrame()
tickers = ['290120', '050120', '151860', '040610', '016250', '121440', '035080',
       '036800', '091590', '039560', '032190', '017650', '007680', '005710',
       '140520', '067990', '088910', '013120', '038060', '012700', '241770',
       '101330', '053700', '023600', '037460', '017480', '038540', '093920',
       '011370', '035890', '011560', '024830', '115570', '036710', '037760',
       '352700', '115480', '127710', '054800', '013310', '306040', '003800',
       '073540', '054940', '009780', '073560', '032940', '030530', '012620',
       '377460', '024800', '119850', '024880', '033290', '052330', '124560',
       '104480', '081150', '106240', '225590', '003380', '023760', '045100',
       '205470']

for ticker in tickers : 
    data = stock.get_market_ohlcv('20240215', '20240315', ticker)
    data = data[['종가']]
    std_dev = data['종가'].rolling(20).std()
    
    data['중심 밴드'] = data['종가'].rolling(20).mean()
    data['이격도'] = (data['종가'] / data['중심 밴드']) * 100
    data = data.dropna()
    
    result_df[ticker] = data['이격도']


result_df = result_df.rename_axis(index='티커')
result_df = result_df.T 

In [18]:
result_df.describe()

티커,2024-03-14,2024-03-15
count,64.000000,64.000000
mean,98.521548,98.567811
std,4.435725,4.398346
min,91.470509,92.055485
25%,95.999769,96.149201
50%,97.845470,97.923120
75%,100.057215,99.885604
max,117.671475,117.827764


In [19]:
re_df = result_df.loc[(result_df.iloc[:, 0] < 98) & (result_df.iloc[:, 1] < 98)]
re_df.shape

(29, 2)

In [30]:
re_df.index

Index(['290120', '050120', '016250', '121440', '091590', '088910', '013120',
       '012700', '241770', '101330', '053700', '023600', '115570', '036710',
       '037760', '115480', '306040', '073540', '054940', '030530', '024800',
       '033290', '124560', '104480', '081150', '106240', '225590', '003380',
       '205470'],
      dtype='object')

In [102]:
import pandas as pd
import json
import requests

ticker = 290120
url = 'https://navercomp.wisereport.co.kr/v2/company/ajax/cF1001.aspx?cmp_cd=290120&fin_typ=4&freq_typ=Y&encparam=SUNzS3VCUDMrOUVJQ0U1Rmo0RlBEZz09&id=bG05RlB6cn'
headers= {
'Referer':'https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd=290120&target=finsum_more',
'User-Agent' :
'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36',
'X-Requested-With' :
'XMLHttpRequest' }

res = requests.get(url=url, headers=headers)
df = pd.read_html(res.text)
df

[    4133  4133.1  4133.2  4133.3  4133.4  4133.5  4133.6  4133.7  4133.8
 0   4133    4133    4133    4133    4133    4133    4133    4133    4133
 1   4133    4133    4133    4133    4133    4133    4133    4133    4133
 2   4133    4133    4133    4133    4133    4133    4133    4133    4133
 3   4133    4133    4133    4133    4133    4133    4133    4133    4133
 4   4133    4133    4133    4133    4133    4133    4133    4133    4133
 5   4133    4133    4133    4133    4133    4133    4133    4133    4133
 6   4133    4133    4133    4133    4133    4133    4133    4133    4133
 7   4133    4133    4133    4133    4133    4133    4133    4133    4133
 8   4133    4133    4133    4133    4133    4133    4133    4133    4133
 9   4133    4133    4133    4133    4133    4133    4133    4133    4133
 10  4133    4133    4133    4133    4133    4133    4133    4133    4133
 11  4133    4133    4133    4133    4133    4133    4133    4133    4133
 12  4133    4133    4133    4133    4

In [105]:
import pandas as pd
import json
import requests

# ticker 변수 설정
ticker = '016250'

# URL 템플릿
url_template = 'https://navercomp.wisereport.co.kr/v2/company/ajax/cF1001.aspx?cmp_cd={}&fin_typ=4&freq_typ=Y&encparam=Wi9Ub01PdFpHRzI0aWJGWERQeFVDZz09&id=aFVlanREZS'

# 대체된 URL 생성
url = url_template.format(ticker)

# Referer 템플릿
referer_template = 'https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd=005930&target=finsum_more'

# 대체된 Referer 생성
referer = referer_template.replace('005930', ticker)

headers = {
    'Referer': referer,
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36',
    'X-Requested-With': 'XMLHttpRequest'
}

res = requests.get(url=url, headers=headers)
df = pd.read_html(res.text)[1]
df = df[[('주요재무정보', '주요재무정보'),
                 ('연간', '2021/12  (IFRS연결)'),
                 ('연간', '2022/12  (IFRS연결)')]]
df = df.iloc[23:28]
df['변화율'] = (df.iloc[:, 2] - df.iloc[:, 1]) / df.iloc[:, 1]
df


주요재무정보                연간                         변화율
    주요재무정보 2021/12  (IFRS연결) 2022/12  (IFRS연결)          
23    부채비율            189.70            295.24  0.556352
24   자본유보율           1985.05           1919.57 -0.032987
25  EPS(원)          20898.00          15620.00 -0.252560
26  PER(배)              2.31              1.82 -0.212121
27  BPS(원)          64409.00          77615.00  0.205033

In [120]:
import pandas as pd
import json
import requests

# 분석할 종목들
tickers = ['290120', '050120', '016250', '121440', '091590', '088910', '013120',
           '012700', '241770', '101330', '053700', '023600', '115570', '036710',
           '037760', '115480', '306040', '073540', '054940', '030530', '024800',
           '033290', '124560', '104480', '081150', '106240', '225590', '003380',
           '205470']

# 결과를 저장할 데이터프레임
result_df = pd.DataFrame(columns=['종목', '변화율'])

# 각 종목에 대해 반복하여 데이터 수집 및 분석 수행
for ticker in tickers:
    # URL 및 Referer 생성
    url_template = 'https://navercomp.wisereport.co.kr/v2/company/ajax/cF1001.aspx?cmp_cd={}&fin_typ=4&freq_typ=Y&encparam=Wi9Ub01PdFpHRzI0aWJGWERQeFVDZz09&id=aFVlanREZS'
    url = url_template.format(ticker)
    referer_template = 'https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd=005930&target=finsum_more'
    referer = referer_template.replace('005930', ticker)
    
    # HTTP 요청
    headers = {
        'Referer': referer,
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36',
        'X-Requested-With': 'XMLHttpRequest'
    }
    res = requests.get(url=url, headers=headers)
    df = pd.read_html(res.text)[1]
    
    # 데이터 전처리 및 분석
    df = df[[('주요재무정보', '주요재무정보'), ('연간', '2021/12  (IFRS연결)'), ('연간', '2022/12  (IFRS연결)')]]
    df = df.iloc[23:28]
    df['변화율'] = (df.iloc[:, 2] - df.iloc[:, 1]) / df.iloc[:, 1]
    
    # 종목별 결과를 데이터프레임에 추가
    df['종목'] = ticker   # '종목' 열에 티커 값 할당
    result_df = pd.concat([result_df, df], ignore_index=True)

result_df

,종목,변화율,"(주요재무정보, 주요재무정보)","(연간, 2021/12 (IFRS연결))","(연간, 2022/12 (IFRS연결))","(변화율, )","(종목, )"
0,NaN,NaN,부채비율,232.47,163.47,-0.296812,290120
1,NaN,NaN,자본유보율,971.08,1339.56,0.379454,290120
2,NaN,NaN,EPS(원),743.00,2066.00,1.780619,290120
3,NaN,NaN,PER(배),8.02,2.61,-0.674564,290120
4,NaN,NaN,BPS(원),5723.00,7447.00,0.301241,290120
...,...,...,...,...,...,...,...
140,NaN,NaN,부채비율,68.67,34.84,-0.492646,205470
141,NaN,NaN,자본유보율,5726.92,10745.64,0.876338,205470
142,NaN,NaN,EPS(원),1158.00,1402.00,0.210708,205470
143,NaN,NaN,PER(배),3.78,2.66,-0.296296,205470


In [126]:
result_df.columns

Index([                       '종목',                       '변화율',
              ('주요재무정보', '주요재무정보'), ('연간', '2021/12  (IFRS연결)'),
       ('연간', '2022/12  (IFRS연결)'),                 ('변화율', ''),
                        ('종목', '')],
      dtype='object')

In [135]:
grouped_df = result_df.groupby(('종목', ''))
grouped_df.head()

,종목,변화율,"(주요재무정보, 주요재무정보)","(연간, 2021/12 (IFRS연결))","(연간, 2022/12 (IFRS연결))","(변화율, )","(종목, )"
0,NaN,NaN,부채비율,232.47,163.47,-0.296812,290120
1,NaN,NaN,자본유보율,971.08,1339.56,0.379454,290120
2,NaN,NaN,EPS(원),743.00,2066.00,1.780619,290120
3,NaN,NaN,PER(배),8.02,2.61,-0.674564,290120
4,NaN,NaN,BPS(원),5723.00,7447.00,0.301241,290120
...,...,...,...,...,...,...,...
140,NaN,NaN,부채비율,68.67,34.84,-0.492646,205470
141,NaN,NaN,자본유보율,5726.92,10745.64,0.876338,205470
142,NaN,NaN,EPS(원),1158.00,1402.00,0.210708,205470
143,NaN,NaN,PER(배),3.78,2.66,-0.296296,205470


In [139]:
df = result_df.drop(columns=['종목', '변화율'])
df.columns

Index([       ('주요재무정보', '주요재무정보'), ('연간', '2021/12  (IFRS연결)'),
       ('연간', '2022/12  (IFRS연결)'),                 ('변화율', ''),
                        ('종목', '')],
      dtype='object')

In [146]:
df = df.rename(columns={('주요재무정보', '주요재무정보'): '재무비율', 
                        ('연간', '2021/12  (IFRS연결)'): '2021/12', 
                        ('연간', '2022/12  (IFRS연결)'): '2022/12', 
                        ('변화율', ''): '변화율', 
                        ('종목', ''): '종목'})

In [147]:
df.columns

Index(['재무비율', '2021/12', '2022/12', '변화율', '종목'], dtype='object')

In [158]:
df

,재무비율,2021/12,2022/12,변화율,종목
0,부채비율,232.47,163.47,-0.296812,290120
1,자본유보율,971.08,1339.56,0.379454,290120
2,EPS(원),743.00,2066.00,1.780619,290120
3,PER(배),8.02,2.61,-0.674564,290120
4,BPS(원),5723.00,7447.00,0.301241,290120
...,...,...,...,...,...
140,부채비율,68.67,34.84,-0.492646,205470
141,자본유보율,5726.92,10745.64,0.876338,205470
142,EPS(원),1158.00,1402.00,0.210708,205470
143,PER(배),3.78,2.66,-0.296296,205470


In [181]:
#부채비율이 감소한 기업 추가 필터링
df.loc[(df['재무비율'] == '부채비율') & (df['변화율'] < 0) & (df['2022/12'] < 200)]['종목'].unique()
temp1 = df.loc[(df['재무비율'] == '부채비율') & (df['변화율'] < 0)]
temp1.shape #18개로 필터링

(18, 5)

In [187]:
df.loc[(df['재무비율'] == 'EPS(원)') & (df['변화율'] > 0)]['종목'].unique()
temp2 = df.loc[(df['재무비율'] == 'EPS(원)') & (df['변화율'] > 0)]
temp2.shape #16개 필터링

(16, 5)

In [194]:
final = pd.merge(temp1, temp2, on='종목')
final['종목'] #10가지 끝

0     290120
1     050120
2     091590
3     101330
4     053700
5     023600
6     115570
7     024800
8     124560
9     106240
10    205470
Name: 종목, dtype: object

In [247]:
graph = pd.merge(df, final, on='종목')
graph = graph[['재무비율', '2021/12', '2022/12', '변화율', '종목']]
graph2 = graph.loc[graph['재무비율'] == 'PER(배)', ['변화율', '종목']]

import plotly.express as px

# 그래프 생성
fig = px.bar(graph2, x='종목', y='변화율', color='변화율', color_continuous_scale='blues')

# 레이아웃 설정
fig.update_layout(
    title='<b>종목별 PER 변화율<b>',
    xaxis_title='종목',
    yaxis_title='변화율',
    yaxis_tickformat="%",
    plot_bgcolor='rgba(0,0,0,0)',
    template='seaborn'
)

# 그래프 표시
fig.show()


In [236]:
cf.help('bar')

BAR
Bar Chart
Supports categories and horizontal bar charts


Parameters:
    bargap : float
        Sets the gap between bars
        	[0,1)
    bargroupgap : float
        Sets the gap between groups
        	[0,1)
    barmode : string
        Bar mode
        	group
        	stack
        	overlay
    categories : string
        Name of the column that contains the categories
    orientation : string
        Sets the orientation of the bars.
        	h
        	v
    sortbars : bool
        Sort bars in descending order


    colors : dict, list or string
        Trace color
        	string : applies to all traces
        	list : applies to each trace in the order specified
        	dict : {column:value} for each column in the dataframe
        values
        	colorname : see cufflinks.colors.cnames
        	hex : '#ffffff'
        	rgb : 'rgb(23,50,23)'
        	rgba : 'rgba(23,50,23,.5)
    colorscale : string
        Color scale name
        If the color is preceded by a minus (-